In [ ]:
import pandas as pd
import time


#˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅˅ cambiar por translate_v2 EN CASO DE USAR PARA PRUEBAS!!!! para hacer los embeddings cuando tengamos ok de gastar, el v3. El v2 es para pruebas, gratis
from google.cloud import translate_v3 as translate
#˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄˄

In [3]:
translate_client = translate.TranslationServiceClient.from_service_account_json(
    '../../../bubbo-dfba0-920a59ed4344.json')

project_id = "bubbo-dfba0"  # ID del proyecto de Bubbo 
location = "global"  

In [ ]:
df = pd.read_csv(r'../../../final2.0.csv', sep=',')  # 274k
df_2 = df.head(50)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274319 entries, 0 to 274318
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    274319 non-null  int64 
 1   ID            274319 non-null  int64 
 2   Genre         274319 non-null  object
 3   CleanTitle    274319 non-null  object
 4   Synopsis      274319 non-null  object
 5   Directors     274319 non-null  object
 6   Cast          274319 non-null  object
 7   Type          274319 non-null  object
 8   PlatformName  274319 non-null  object
dtypes: int64(2), object(7)
memory usage: 18.8+ MB


In [6]:
def translate_text(text, project_id, location, target_language_code):
    parent = f"projects/{project_id}/locations/{location}"
    response = translate_client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",
            "target_language_code": target_language_code,
        }
    )
    # Accede al primer elemento de la lista y luego a su atributo translated_text
    return response.translations[0].translated_text



def translate_df(df, columnas, project_id, location):
    aux_df = df.copy()
    parent = f"projects/{project_id}/locations/{location}"

    for col in columnas:
        if col in df.columns:
            aux_df[f'translated_{col}'] = df[col].apply(lambda x: translate_text(x, project_id, location, 'en') 
                                                        if isinstance(x, str) and 
                                                        translate_client.detect_language(request={"parent": parent, "content": x}).languages[0].language_code != 'en' 
                                                        else x)
    return aux_df


In [ ]:
columnas_a_traducir = ['CleanTitle', 'Synopsis', 'Genres']
df_traducido = translate_df(df_2, columnas_a_traducir, project_id, location)